In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


def train_model(X_train, y_train, params):
    """
    Train a logistic regression model with the given parameters.
    
    Args:
        X_train (array-like): Training features.
        y_train (array-like): Training labels.
        params (dict): Hyperparameters for the logistic regression model.
        
    Returns:
        LogisticRegression: Trained logistic regression model.
    """
    # train the logistic regression model
    lr = LogisticRegression(**params)
    lr.fit(X_train, y_train)
    # Predict on the test set
    y_pred = lr.predict(X_test)
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    # Create a DataFrame to hold the metrics
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "model": lr
    }

In [2]:

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 43,
}

# Train the model and get the metrics
metrics = train_model(X_train, y_train, params)

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [3]:
metrics

{'accuracy': 1.0,
 'precision': 1.0,
 'recall': 1.0,
 'f1_score': 1.0,
 'model': LogisticRegression(max_iter=1000, multi_class='auto', random_state=43)}

In [4]:
import mlflow
from mlflow.models import infer_signature
# Set our tracking server uri for logging
mlflow.set_tracking_uri("sqlite:///mlflow.db")
# Create a new MLflow Experiment
mlflow.set_experiment("LogisticRegression")

2025/05/26 17:25:13 INFO mlflow.tracking.fluent: Experiment with name 'LogisticRegression' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/gbemidebe/Documents/GitHub/learning-mlops-zoomcamp/02-experiment-tracking/examples/case04/mlruns/3', creation_time=1748298313573, experiment_id='3', last_update_time=1748298313573, lifecycle_stage='active', name='LogisticRegression', tags={}>

In [9]:
# Start an MLflow run
with mlflow.start_run():
    # Define the model hyperparameters
    params = {
        "solver": "lbfgs",
        "max_iter": 50,
        "multi_class": "auto",
        "random_state": 123,
    }
    # Log the hyperparameters
    mlflow.log_params(params)

    # Train the model
    results = train_model(X_train, y_train, params)

    # Log the loss metric
    mlflow.log_metric("accuracy", results["accuracy"])
    mlflow.log_metric("precision", results["precision"])
    mlflow.log_metric("recall", results["recall"])
    mlflow.log_metric("f1_score", results["f1_score"])

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, results['model'].predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=results["model"],
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptool

In [7]:
# load the model back
logged_model = 'runs:/26ce6c6dccf74635aa74c47b70e0baec/iris_model'
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [8]:
loaded_model.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [10]:
mlflow.search_experiments()

[<Experiment: artifact_location='/Users/gbemidebe/Documents/GitHub/learning-mlops-zoomcamp/02-experiment-tracking/examples/case04/mlruns/3', creation_time=1748298313573, experiment_id='3', last_update_time=1748298313573, lifecycle_stage='active', name='LogisticRegression', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748296312074, experiment_id='0', last_update_time=1748296312074, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
# Create a new MLflow Experiment
mlflow.set_experiment("XGBRegression")

2025/05/26 18:14:22 INFO mlflow.tracking.fluent: Experiment with name 'XGBRegression' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/gbemidebe/Documents/GitHub/learning-mlops-zoomcamp/02-experiment-tracking/examples/case04/mlruns/4', creation_time=1748301262517, experiment_id='4', last_update_time=1748301262517, lifecycle_stage='active', name='XGBRegression', tags={}>

In [18]:
from xgboost import XGBClassifier

In [19]:
XGBClassifier.fit??

Signature:
XGBClassifier.fit(
    self,
    X: Any,
    y: Any,
    *,
    sample_weight: Optional[Any] = None,
    base_margin: Optional[Any] = None,
    eval_set: Optional[Sequence[Tuple[Any, Any]]] = None,
    verbose: Union[bool, int, NoneType] = True,
    xgb_model: Union[xgboost.core.Booster, str, xgboost.sklearn.XGBModel, NoneType] = None,
    sample_weight_eval_set: Optional[Sequence[Any]] = None,
    base_margin_eval_set: Optional[Sequence[Any]] = None,
    feature_weights: Optional[Any] = None,
) -> 'XGBClassifier'
Docstring:
Fit gradient boosting classifier.

Note that calling ``fit()`` multiple times will cause the model object to be
re-fit from scratch. To resume training from a previous checkpoint, explicitly
pass ``xgb_model`` argument.

Parameters
----------
X :
    Input feature matrix. See :ref:`py-data` for a list of supported types.

    When the ``tree_method`` is set to ``hist``, internally, the
    :py:class:`QuantileDMatrix` will be used instead of the :py:class

In [25]:
params

{'solver': 'lbfgs', 'max_iter': 50, 'multi_class': 'auto', 'random_state': 123}

In [26]:
md = XGBClassifier(
    **params)

In [27]:
md.fit(X=X_train, y=y_train, eval_set= [(X_test, y_test)], 
        verbose=False)

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [18:22:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "max_iter", "multi_class", "solver" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_iter=50, max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_class='auto',
              multi_strategy=None, n_estimators=None, ...)

In [ ]:
from sklearn.metrics import root_mean_squared_error, r2_score

In [40]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        # initialize the model
        md = XGBClassifier(**params)
        # Train the model
        md.fit(X_train, y_train, eval_set= [(X_test, y_test)], verbose=False)
        # Log the model
        y_pred = md.predict(X_test)
        mlflow.log_metric("rmse", root_mean_squared_error(y_test, y_pred))
        mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
        mlflow.log_metric('precision', precision_score(y_test, y_pred, average='weighted'))
        mlflow.log_metric('recall', recall_score(y_test, y_pred, average='weighted'))
        mlflow.log_metric('f1_score', f1_score(y_test, y_pred, average='weighted'))
        mlflow.xgboost.log_model(md, artifact_path="xgboost_model")

    return {
            'loss': root_mean_squared_error(y_test, y_pred), 
            'status': STATUS_OK}

In [41]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 1, 100, 10)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [18:27:19] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)



 10%|█         | 1/10 [00:02<00:21,  2.39s/trial, best loss: 0.0]

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [18:27:21] WARNING: /Users/ru

 20%|██        | 2/10 [00:04<00:16,  2.02s/trial, best loss: 0.0]

...

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [18:27:34] WARNING: /Users/ru

 90%|█████████ | 9/10 [00:16<00:01,  1.77s/trial, best loss: 0.0]

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [18:27:35] WARNING: /Users/ru

100%|██████████| 10/10 [00:18<00:00,  1.86s/trial, best loss: 0.0]


/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

/opt/anaconda3/envs/datatalk/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(

